In [4]:
import inspect
import json
import typing
from enum import Enum
from typing import Any, Callable, Dict, List, Optional, Type, get_type_hints, get_origin, get_args

from griffe import Module
from griffe.dataclasses import Function
from griffe.loader import GriffeLoader
from pydantic import BaseModel

# Type mappings from Python/Pydantic types to Gemini schema types
TYPE_MAPPING = {
    str: {"type": "STRING"},
    int: {"type": "INTEGER"},
    float: {"type": "NUMBER"},
    bool: {"type": "BOOLEAN"},
    list: {"type": "ARRAY"},
    dict: {"type": "OBJECT"},
    # Add more mappings as needed
}

def get_type_schema(type_hint: Type) -> Dict[str, Any]:
    """Convert a Python type hint to a Gemini schema type."""
    # Handle None/Optional types
    if type_hint is type(None) or type_hint is None:
        return {"type": "NULL"}
    
    # Handle basic types
    if type_hint in TYPE_MAPPING:
        return TYPE_MAPPING[type_hint]
    
    # Handle Optional[...] types
    origin = get_origin(type_hint)
    if origin is Union:
        args = get_args(type_hint)
        if type(None) in args or None in args:
            # This is an Optional[...] type
            # Find the non-None type
            non_none_args = [arg for arg in args if arg is not type(None) and arg is not None]
            if len(non_none_args) == 1:
                schema = get_type_schema(non_none_args[0])
                schema["nullable"] = True
                return schema
    
    # Handle List[...] types
    if origin is list:
        args = get_args(type_hint)
        if args:
            return {
                "type": "ARRAY",
                "items": get_type_schema(args[0])
            }
        return {"type": "ARRAY"}
    
    # Handle Dict[...] types
    if origin is dict:
        return {"type": "OBJECT"}
    
    # Handle Enum types
    if isinstance(type_hint, type) and issubclass(type_hint, Enum):
        return {
            "type": "STRING",
            "format": "enum",
            "enum": [e.value for e in type_hint]
        }
    
    # Handle Pydantic models
    if isinstance(type_hint, type) and issubclass(type_hint, BaseModel):
        schema = type_hint.model_json_schema()
        return convert_schema_to_gemini(schema)
    
    # Default to string if we can't determine the type
    return {"type": "STRING"}

def convert_schema_to_gemini(schema: Dict[str, Any]) -> Dict[str, Any]:
    """Convert a JSON Schema to Gemini schema format."""
    gemini_schema = {}
    
    # Handle type
    if "type" in schema:
        type_mapping = {
            "string": "STRING",
            "number": "NUMBER",
            "integer": "INTEGER",
            "boolean": "BOOLEAN",
            "array": "ARRAY",
            "object": "OBJECT",
            "null": "NULL"
        }
        gemini_schema["type"] = type_mapping.get(schema["type"], schema["type"])
    
    # Handle description
    if "description" in schema:
        gemini_schema["description"] = schema["description"]
    elif "title" in schema:
        gemini_schema["description"] = schema["title"]
    
    # Handle enum
    if "enum" in schema:
        gemini_schema["enum"] = schema["enum"]
        if "type" not in gemini_schema:
            gemini_schema["type"] = "STRING"
            gemini_schema["format"] = "enum"
    
    # Handle properties for objects
    if "properties" in schema:
        gemini_schema["properties"] = {}
        for prop_name, prop_schema in schema["properties"].items():
            gemini_schema["properties"][prop_name] = convert_schema_to_gemini(prop_schema)
        
        # Add propertyOrdering
        gemini_schema["propertyOrdering"] = list(schema["properties"].keys())
    
    # Handle required fields
    if "required" in schema:
        gemini_schema["required"] = schema["required"]
    
    # Handle array items
    if "items" in schema:
        gemini_schema["items"] = convert_schema_to_gemini(schema["items"])
    
    # Handle nullable
    if schema.get("nullable", False):
        gemini_schema["nullable"] = True
    
    return gemini_schema

def function_to_gemini_schema(func: Callable) -> Dict[str, Any]:
    """Convert a Python function to Gemini function declaration schema."""
    # Get function name and docstring
    func_name = func.__name__
    func_doc = inspect.getdoc(func) or ""
    
    # Create the basic function declaration
    function_declaration = {
        "name": func_name,
        "description": func_doc
    }
    
    # Get type hints and signature
    type_hints = get_type_hints(func)
    signature = inspect.signature(func)
    
    # Check if function has parameters
    if signature.parameters:
        # Create parameters schema
        parameters_schema = {
            "type": "OBJECT",
            "properties": {}
        }
        
        required = []
        
        for param_name, param in signature.parameters.items():
            # Skip self or cls parameters
            if param_name in ("self", "cls"):
                continue
            
            # Get parameter type hint
            param_type = type_hints.get(param_name, Any)
            
            # Convert type hint to schema
            param_schema = get_type_schema(param_type)
            
            # Add description from docstring if available
            # This assumes a Google-style docstring with Args: section
            docstring_lines = func_doc.split("\n")
            param_doc = ""
            in_args_section = False
            current_param = None
            
            for line in docstring_lines:
                line = line.strip()
                if line.lower().startswith("args:"):
                    in_args_section = True
                    continue
                elif line.lower().startswith(("returns:", "raises:", "yields:")) or not line:
                    in_args_section = False
                    
                if in_args_section and line:
                    if line.startswith(param_name + ":") or line.startswith(param_name + " "):
                        current_param = param_name
                        param_doc = line[len(param_name) + 1:].strip()
                    elif current_param == param_name and line:
                        param_doc += " " + line
            
            if param_doc:
                param_schema["description"] = param_doc
            
            # Add parameter to schema
            parameters_schema["properties"][param_name] = param_schema
            
            # Check if parameter is required
            if param.default is inspect.Parameter.empty:
                required.append(param_name)
        
        # Add required parameters
        if required:
            parameters_schema["required"] = required
        
        # Add parameters to function declaration
        function_declaration["parameters"] = parameters_schema
    
    return function_declaration

def module_to_gemini_schema(module_path: str) -> Dict[str, Any]:
    """Convert all functions in a module to Gemini function declarations schema."""
    # Load the module using griffe
    loader = GriffeLoader()
    module = loader.load(module_path)
    
    function_declarations = []
    
    # Process all functions in the module
    for obj_name, obj in module.objects.items():
        if isinstance(obj, Function):
            try:
                # Get the actual Python function
                import_path = f"{module_path}.{obj_name}"
                module_parts = import_path.split('.')
                mod = __import__(module_parts[0])
                for part in module_parts[1:]:
                    mod = getattr(mod, part)
                
                # Convert function to schema
                function_schema = function_to_gemini_schema(mod)
                function_declarations.append(function_schema)
            except (ImportError, AttributeError) as e:
                print(f"Error processing function {obj_name}: {e}")
    
    return {"function_declarations": function_declarations}

# Example usage with custom functions
def enable_lights():
    """Turn on the lighting system."""
    pass

def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work.
    
    Args:
        rgb_hex: The light color as a 6-digit hex string, e.g. ff0000 for red.
    """
    pass

def stop_lights():
    """Turn off the lighting system."""
    pass

# Convert individual functions
functions = [enable_lights, set_light_color, stop_lights]
function_declarations = [function_to_gemini_schema(func) for func in functions]
schema = {"function_declarations": function_declarations}

print(json.dumps(schema, indent=2))

# Example with a module (uncomment to use)
# schema = module_to_gemini_schema('your_module_name')
# print(json.dumps(schema, indent=2))